In [1]:
from squirro_client import SquirroClient

In [2]:
CLUSTER='http://172.16.213.160:81/'
TOKEN='6cd91079f28105bafdfa525b546120eda0d53ac2f5227d3fb9d3be2a0404dcd7a86c4e7a902da2c9d7493a4a6dcba75f5d81d7354bd7e2f6414b415bae284108'
PROJECT_ID='fvQAaO0-RHqN0NbLkQGcnw'

# Connect to Squirro

In [3]:
client = SquirroClient(client_id=None, client_secret=None, cluster=CLUSTER)
client.authenticate(refresh_token=TOKEN)

## Take a Look at the data

In [7]:
for item in client.query(project_id=PROJECT_ID, query='*', count=10)['items']:
    
    first_name = item['keywords'].get('first_name', ['?'])[0]
    gender = item['keywords'].get('gender', ['?'])[0]
    
    print u'{gender} - {name}'.format(
        name=first_name, 
        gender=gender)

M - John
M - Michael
F - Rebecca
M - Emil
M - Keith
M - Jingdong
M - Scott
M - David
M - Oliver
M - Paul


# Create the Workflow

In [12]:
client.get_machinelearning_workflows(project_id=PROJECT_ID)

{u'machinelearning_workflows': []}

In [13]:
workflow = {
  "dataset": {
    "train": {
      "query_string": "gender:*"
    },
    "process": {
      "query_string": "*"
    }
  },
  "pipeline": [
    {
      "step": "loader",
      "type": "squirro_query",
      "fields": [
        "keywords.gender",
        "keywords.first_name"
      ]
    },
    {
      "step": "filter",
      "type": "empty",
      "fields": [
        "keywords.first_name",
        "keywords.gender"
      ]
    },
    {
      "step": "filter",
      "type": "join",
      "input_field": "keywords.first_name",
      "output_field": "keywords.first_name"
    },
    {
      "step": "normalizers",
      "types": [
        "html",
        "punctuation",
        "lowercase",
        "character"
      ],
      "fields": [
        "keywords.first_name"
      ]
    },
    {
      "step": "tokenizer",
      "type": "spaces",
      "fields": [
        "keywords.first_name"
      ]
    },
    {
      "step": "embedder",
      "type": "dictionary",
      "batch_size": 1024,
      "input_field": "keywords.first_name",
      "output_field": "embedded_text"
    },
    {
      "step": "checkpoint",
      "type": "disk",
      "do_randomize": True,
      "batch_size": 1
    },
    {
      "step": "classifier",
      "type": "seq2one",
      "batch_size": 1024,
      "dict_name": "dictionary",
      "dropout_fraction": 0.5,
      "embedding_dim": 50,
      "explanation_field": "explanantion",
      "input_field": "embedded_text",
      "label_field": "keywords.gender",
      "labels": [
        "M",
        "F"
      ],
      "max_sequence_length": 1000,
      "mini_batch_size": 64,
      "n_epochs": 10,
      "output_field": "keywords.pred_gender"
    },
    {
      "step": "saver",
      "type": "squirro_item",
      "batch_size": 1000,
      "fields": [
        "keywords.pred_gender"
      ]
    }
  ]
}

In [14]:
# Upload the workflow
workflow_id = client.new_machinelearning_workflow(name='gender_predictor', project_id=PROJECT_ID, config=workflow).get('id')

In [15]:
client.get_machinelearning_workflow(project_id=PROJECT_ID, ml_workflow_id=workflow_id)

{u'machinelearning_workflow': {u'config': {u'dataset': {u'process': {u'query_string': u'*'},
    u'train': {u'query_string': u'gender:*'}},
   u'pipeline': [{u'fields': [u'keywords.gender', u'keywords.first_name'],
     u'step': u'loader',
     u'type': u'squirro_query'},
    {u'fields': [u'keywords.first_name', u'keywords.gender'],
     u'step': u'filter',
     u'type': u'empty'},
    {u'input_field': u'keywords.first_name',
     u'output_field': u'keywords.first_name',
     u'step': u'filter',
     u'type': u'join'},
    {u'fields': [u'keywords.first_name'],
     u'step': u'normalizers',
     u'types': [u'html', u'punctuation', u'lowercase', u'character']},
    {u'fields': [u'keywords.first_name'],
     u'step': u'tokenizer',
     u'type': u'spaces'},
    {u'batch_size': 1024,
     u'input_field': u'keywords.first_name',
     u'output_field': u'embedded_text',
     u'step': u'embedder',
     u'type': u'dictionary'},
    {u'batch_size': 1,
     u'do_randomize': True,
     u'step': u'c

# Create Training Job

In [20]:
client.get_machinelearning_jobs(ml_workflow_id=workflow_id, project_id=PROJECT_ID)

{u'machinelearning_jobs': [{u'created_at': u'2018-04-21T21:47:24',
   u'error_count': 0,
   u'healthy': False,
   u'id': u'sToW68BqTkm1Es4Y1Kf6Qg',
   u'last_error': None,
   u'last_error_at': None,
   u'last_success_at': None,
   u'ml_workflow_id': u'J2WH8v4oRwKiaOCKLMKs2Q',
   u'modified_at': u'2018-04-21T21:47:24',
   u'next_run_time_at': u'2018-04-21T22:02:24',
   u'total_runs': 0,
   u'type': u'training'}]}

In [17]:
job_id = client.new_machinelearning_job(project_id=PROJECT_ID, ml_workflow_id=workflow_id, type='training').get('id')

# Reset

In [58]:
client.delete_machinelearning_job(ml_job_id=job_id, ml_workflow_id=workflow_id, project_id=PROJECT_ID)

{}

In [61]:
client.delete_machinelearning_workflow(project_id=PROJECT_ID, ml_workflow_id=workflow_id)

NotFoundError: (404, u'Not Found')